### Notebook Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import find_dotenv
import pathlib as pb
import sys


# Add `ape` package to SYS path
sys.path.append(str(pb.Path(find_dotenv()).parent))

### Imports

In [3]:
from typing import Dict, cast
import re
import torch
from torch.utils.data import Dataset, Subset
from torch.utils.data import DataLoader, random_split
import evaluate as eval
import lightning as lit
from lightning.pytorch.loggers import MLFlowLogger
from dataclasses import asdict
from transformers import AlbertTokenizer
from collections import defaultdict, namedtuple
from transformers import AutoModel, AutoTokenizer, XLMRobertaForCausalLM
from functools import partial

In [4]:
from onmt.onmt.modules.position_ffn import ActivationFunction
from onmt.onmt.translate.translator import GeneratorLM

In [5]:
import ape
from ape.data.types import DataSplit
from ape.data.types import APETripletDict
from ape.eval.metrics import APEMetrics
from ape.data.dataset import APEDataset
from ape.data.transform import Tokenize, HFTokenizer
from ape.model.mst import MultiSourceTransformerCausalLM
from ape.model.encoders import MultiSourceTransformerEncoder
from ape.model.decoders import MultiSourceTransformerDecoder
from ape.light.causal_lm import MultiSourceCausalLMLightningModule

### Data & Metrics

In [6]:
# Load TER, chrF and BLEU
metrics = APEMetrics(cache_dir=ape.HF_CACHE_DIR)

# Load APE Dataset (Original + Synthetic)
ds_test = APEDataset(path=ape.DATA_DIR, split='test')
ds_train = APEDataset(path=ape.DATA_DIR, split='train')
ds_train, ds_valid = random_split(ds_train, lengths=[0.99, 0.01], generator=ape.gen_torch)

# Aggregate all subsets into a single object
ds: Dict[DataSplit, Dataset[APETripletDict[str]]] = {
    'train': ds_train,
    'valid': ds_valid,
    'test': ds_test,
}

In [7]:
# Choose encoders for each input type
encoder_type_src = 'roberta-base'
encoder_type_mt = 'l3cube-pune/marathi-roberta'

# Load source and target tokenizers
hf_tokenizer_src = AutoTokenizer.from_pretrained(encoder_type_src,
                                                                                 use_fast=True,
                                                                                 padding_side='right',
                                                                                 truncation_side='right',
                                                                                 cache_dir=ape.HF_CACHE_DIR / 'tokenizers')
hf_tokenizer_mt = AutoTokenizer.from_pretrained(encoder_type_mt,
                                                                                use_fast=True,
                                                                                padding_side='right',
                                                                                truncation_side='right',
                                                                                cache_dir=ape.HF_CACHE_DIR / 'tokenizers')

# Wrap and customize HF Tokenizers
max_seq_len = 512
tokenizer_src = HFTokenizer(hf_tokenizer_src, source_prefix='src', max_length=max_seq_len)
tokenizer_mt = HFTokenizer(hf_tokenizer_mt, source_prefix='mt', max_length=max_seq_len)
tokenizer_pe = HFTokenizer(hf_tokenizer_mt, source_prefix='pe', max_length=max_seq_len)
tokenize = Tokenize([tokenizer_src, tokenizer_mt, tokenizer_pe])

# Use same settings across all splits
DefaultDataLoader = partial(DataLoader,
                            collate_fn=tokenize,
                            num_workers=ape.WORKERS,
                            batch_size=ape.BATCH_SIZE,
                            prefetch_factor=ape.PREFETCH_FACTOR)

# Aggregate all dataloaders into a single object
dl: Dict[DataSplit, DataLoader] = {
    'train': DefaultDataLoader(ds['train']),
    'valid': DefaultDataLoader(ds['valid']),
    'test': DefaultDataLoader(ds['test']),
}

### Models

In [8]:
model = MultiSourceCausalLMLightningModule(
    bos_token_id=hf_tokenizer_mt.bos_token_id,
    encoder_type_src=encoder_type_src,
    encoder_type_mt=encoder_type_mt,
    tokenizer_mt=hf_tokenizer_mt,
    block_size=max_seq_len,
    temperature=1.0,
    do_sample=True,
    top_k=8,
)

logger = MLFlowLogger(
    experiment_name='Automatic Post-Editing',
    tracking_uri=ape.MLFLOW_TRACKING_URI,
    tags={ 'test': 'true', },
)

trainer = lit.Trainer(
    logger=logger,
    limit_val_batches=200,
    val_check_interval=5_000,
    accumulate_grad_batches=1,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Training

In [9]:
trainer.fit(model, train_dataloaders=dl['train'], val_dataloaders=dl['valid'])

/home/invokariman/.cache/pypoetry/virtualenvs/ape-zcZ_0igR-py3.11/lib/python3.11/site-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                           | Params
---------------------------------------------------------
0 | model | MultiSourceTransformerCausalLM | 629 M 
---------------------------------------------------------
629 M     Trainable params
0         Non-trainable params
629 M     Total params
2,519.085 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/invokariman/.cache/pypoetry/virtualenvs/ape-zcZ_0igR-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/home/invokariman/.cache/pypoetry/virtualenvs/ape-zcZ_0igR-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/home/invokariman/.cache/pypoetry/virtualenvs/ape-zcZ_0igR-py3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [54]:
torch.save(model.state_dict(), ape.ROOT_DIR / '..'/ 'ckpt' / 'weights.pt')

### Manual Evaluation

In [48]:
# Retrieve a batch of examples
batch = next(iter(dl['test']))

# Apply Automatic Post-Editing
post_edit = model.model.post_edit(
    src_input_ids=batch['src_input_ids'].to(ape.DEVICE),
    mt_input_ids=batch['mt_input_ids'].to(ape.DEVICE),
    src_attn_mask=batch['src_attention_mask'].to(ape.DEVICE),
    mt_attn_mask=batch['mt_attention_mask'].to(ape.DEVICE),
    bos_token_id=hf_tokenizer_mt.bos_token_id,
    temperature=1.0,
    do_sample=True,
    top_k=10,
)

In [49]:
# Decode the SRC
hf_tokenizer_src.batch_decode(batch['src_input_ids'])

['<s>Section  A  will  be  of  Multiple-Choice  Questions (MCQs)  and  Section  B  will  contain  Questions  whose  answers  are  to  be  filled  in  as  a numerical value.</s>',
 '<s>God can be achieved only through complete devotion, he preached.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>Itchy red skin (where you had the injection) – apply a soothing cream such as a moisturiser, anti-itch cream or lotion.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>If you or your child has nf2, your clinical team will pass information about you/your child on to the national congenital anomaly and rare disease registration service (ncardrs).</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']

In [50]:
# Decode the MT
hf_tokenizer_mt.batch_decode(batch['mt_input_ids'])

["<s> 'अ' विभागात बहुपर्यायी प्रश्न असतील आणि 'ब' विभागात संख्यात्मक मूल्याचे प्रश्न असतील.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>",
 '<s> "पूर्ण भक्तीनेच देव प्राप्त होऊ शकतो," असे त्यांनी प्रचार केले.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> लाल त्वचा (जिथे तुम्हाला इंजेक्शन देण्यात आले आहे)-मॉइस्चरायझर, अँटी-इच क्रीम किंवा लोशन सारख्या शामक क्रीम लावा.</s>',
 '<s> समुद्राच्या पातळीवर, बॅरॅक, तुरुंग, बंदुकीच्या पावडरसाठी साठवण कक्ष, राहण्याची खोली आणि एक चॅपल देखील आहे.</s><pad><pad>']

In [51]:
# Decode the PE
hf_tokenizer_mt.batch_decode(post_edit)

['<s> यानंतर त्यांना त्यांच्यासोबत लग्न करण्यात आलं.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> त्यामध्ये १.५ कोटींहून अधिक मर्यादित आहेत.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> याप्रकरणी चौकशी करण्यात आले असून, पोलिसांकडून कारवाई सुरू आहे.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>.. हे ध्येय आहे, पण, त्या मर्यादित आहे की, जर तुम्हाला तुमच्या मते, तुमच्यावर तुमच्या डॉक्टरांशी बोला.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>